# Введение в методы сбора и обработки данных
## Добашнее задание к третьему уроку "Парсинг HTML"

### Задача 1
Необходимо собрать информацию о вакансиях на должность программиста или разработчика с сайта job.ru или hh.ru. (Можно с обоих сразу) Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе:

* Наименование вакансии, 
* Предлагаемую зарплату 
* Ссылку на саму вакансию

#### Решение

_job.ru полностью переехал на hh.ru, так что есть для использования только hh.ru_

In [1]:
import requests
import pandas as pd
from lxml import html
import re

Зададим параметры к запросу

In [2]:
def get_vacancy_df(text='"программист" OR "разработчик"'):
    """Возвращает датасет с данными о вакансиях"""
    
    hh_link = 'https://hh.ru/search/vacancy'
    headers = {'accept': '*/*',
           'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:68.0)' 
                        'Gecko/20100101 Firefox/68.0'}
    
    page_num = None
    params = {
        'clusters': 'true',
        'enable_snippets': 'true',
        'only_with_salary': 'true',
        'currency': "RUR",
        'text': text,
        "search_field": "name",
        'showClusters': 'true',
        'area': 1,
        'page': page_num
    }

    all_page_req = []
    session = requests.Session()

    # Запрос к сайту и получение ответа (первые 5 страниц)
    for page_num in range(5):
        request = session.get(hh_link, params=params, headers=headers)

        root = html.fromstring(request.text)

        results_list = root.xpath("//a[contains(@class, 'bloko-link HH-LinkModifier')]/text() "
                                          "| //div[contains(@class, 'vacancy-serp-item__compensation')]/text()"
                                          "| //a[contains(@class, 'bloko-link HH-LinkModifier')]/@href")
        all_page_req.extend(results_list)

    ## Обработка полученных данных
    data_list = []
    id_mask = 'vacancy\/([0-9]*)\?' # Регулярное выражение для получения id

    # Формирование json
    for i in range(0, len(all_page_req), 3):
        data_list.append({
            'index': int(re.search(id_mask, all_page_req[i])[1]),
            'name': all_page_req[i+1],
            'salary': all_page_req[i+2].replace(u'\xa0', u''),
            'link': all_page_req[i].split('?')[0]
        })

    return pd.DataFrame(data_list).set_index('index')[['name', 'salary', 'link']]

Вызовим функцию и проверим результат

In [3]:
df = get_vacancy_df('"программист" OR "разработчик"')

In [4]:
df.head(5)

,name,salary,link
index,,,
32543357,Программист С++ (графика),80000-120000 руб.,https://hh.ru/vacancy/32543357
32946587,Старший программист-разработчик PHP,200000-200000 руб.,https://hh.ru/vacancy/32946587
32842466,Java - разработчик (Middle),от 90000 руб.,https://hh.ru/vacancy/32842466
32379189,JavaScript разработчик (ДИТ HR),от 230000 руб.,https://hh.ru/vacancy/32379189
32323917,Программист С++,60000-100000 руб.,https://hh.ru/vacancy/32323917


### Задача 2
Доработать приложение таким образом, чтобы можно было искать разработчиков на разные языки программирования (Например Python, Java, C++)

#### Решение
В принципе, всё готово, обернем лишь запрос к пользователю и вызовим:

In [5]:
def get_vacancy_df_by_lang():

    lang = input('Какой язык разработки вас интересует?: ')

    return get_vacancy_df(f'"{lang}" AND "разработчик"')

In [6]:
df = get_vacancy_df_by_lang()

Какой язык разработки вас интересует?: Python


In [7]:
df.head(15)

,name,salary,link
index,,,
32438211,Python разработчик,от 120000 руб.,https://hh.ru/vacancy/32438211
32740537,Программист Python (в Ригу),2000-3000 EUR,https://hh.ru/vacancy/32740537
32923751,Python Backend разработчик,120000-160000 руб.,https://hh.ru/vacancy/32923751
31240024,"Младший разработчик веб-скрейперов (Python, Se...",80000-100000 руб.,https://hh.ru/vacancy/31240024
32680147,Python-разработчик,до 200000 руб.,https://hh.ru/vacancy/32680147
32433718,Back-end разработчик (Python),80000-120000 руб.,https://hh.ru/vacancy/32433718
32528796,Python Developer,от 120000 руб.,https://hh.ru/vacancy/32528796
32690277,"Backend Developer (Python, Django)",180000-220000 руб.,https://hh.ru/vacancy/32690277
32954824,Разработчик Python,100000-180000 руб.,https://hh.ru/vacancy/32954824
